# Conformal Prediction
> Tutorial on how to train neuralforecast models and obtain prediction intervals using the conformal prediction methods

Conformal prediction intervals use cross-validation on a point forecaster model to generate the intervals. This means that no prior probabilities are needed, and the output is well-calibrated. No additional training is needed, and the model is treated as a black box. The approach is compatible with any model.

In this notebook, we demonstrate how to obtain intervals using the conformal prediction methods.

## Load libraries

In [ ]:
import logging
import os
import tempfile

import numpy as np
import pandas as pd

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from utilsforecast.losses import mae, rmse, smape
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic

In [ ]:
logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)
os.environ['NIXTLA_ID_AS_COL'] = '1'

## Data

We simply use AirPassengers dataset for the demostration of conformal predictions.


In [ ]:
AirPassengersPanel_train = AirPassengersPanel[AirPassengersPanel['ds'] < AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test = AirPassengersPanel[AirPassengersPanel['ds'] >= AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test['y'] = np.nan
AirPassengersPanel_test['y_[lag12]'] = np.nan

You can also create this directory structure with a spark dataframe using the following:

In [ ]:
"""
spark.conf.set("spark.sql.parquet.outputTimestampType", "TIMESTAMP_MICROS")
(
  spark_df
  .repartition(id_col)
  .sortWithinPartitions(id_col, time_col)
  .write
  .partitionBy(id_col)
  .parquet(out_dir)
)
"""

The DataLoader class still expects the static data to be passed in as a single DataFrame with one row per timeseries.

In [ ]:
static = AirPassengersStatic.rename(columns={'unique_id': 'id_col'})
static

## Model training

We now train a NHITS model on the above dataset. 
It is worth noting that NeuralForecast currently does not support scaling when using this DataLoader. If you want to scale the timeseries this should be done before passing it in to the `fit` method.

In [ ]:
horizon = 12
stacks = 3
models = [NHITS(input_size=5 * horizon,
                h=horizon,
                futr_exog_list=['trend', 'y_[lag12]'],
                stat_exog_list=['airline1', 'airline2'],
                max_steps=100,
                stack_types = stacks*['identity'],
                n_blocks = stacks*[1],
                mlp_units = [[256,256] for _ in range(stacks)],
                n_pool_kernel_size = stacks*[1])]
nf = NeuralForecast(models=models, freq='ME')
nf.fit(df=files_list, static_df=static, id_col='id_col')

## Forecasting

When working with large datasets, we need to provide a single DataFrame containing the input timesteps of all the timeseries for which wish to generate predictions. If we have future exogenous features, we should also include the future values of these features in the separate `futr_df` DataFrame. 

For the below prediction we are assuming we only want to predict the next 12 timesteps for Airline2.

In [ ]:
valid_df = valid[valid['id_col'] == 'Airline2']
# we set input_size=60 and horizon=12 when fitting the model
pred_df = valid_df[:60]
futr_df = valid_df[60:72]
futr_df = futr_df.drop(["y"], axis=1)

predictions = nf.predict(df=pred_df, futr_df=futr_df, static_df=static)

In [ ]:
#| hide
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
# only select those from NHITS to show
preds = preds.reset_index(drop=True)
dropped = [x for x in preds.columns if 'StemGNN' in x or 'RNN' in x]
preds = preds.drop(columns=dropped)

plot_df = pd.concat([AirPassengersPanel_train, preds]).set_index('ds')

plot_df[plot_df['unique_id']=='Airline1'].drop(['unique_id','trend','y_[lag12]'], axis=1).plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

## Caveat

One caveat to note is that we do not support the conformalize quantiled prediction outputs computed by loss functions such as
 * [MQLoss](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#multi-quantile-loss-mqloss)
 * [DistributionLoss](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#distributionloss)
 * [PMM](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#poisson-mixture-mesh-pmm)
 * [GMM](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#gaussian-mixture-mesh-gmm)
 * [NBMM](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#negative-binomial-mixture-mesh-nbmm)
 * [HuberQLoss](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#huberized-quantile-loss)
 * [HuberMQLoss](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#huberized-mqloss)
 * [QuantileLoss](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#quantile-loss)
 * [IQLoss](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#implicit-quantile-loss-iqloss)
 * [sCRPS](https://nixtlaverse.nixtla.io/neuralforecast/losses.pytorch.html#scaled-continuous-ranked-probability-score-scrps)